<a href="https://colab.research.google.com/github/padolekrushna/Prompt-Engineering-Class/blob/main/HOMEWORK_3_Word_Embeddings_and_RAG_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

link to google colab : https://colab.research.google.com/drive/14sWo11mB8gFwq3elZp9Cbw3CvghiSdly?usp=sharing

#Part 1: Word Embedding Arithmetic

##Step 1: Load the BERT Model and Tokenizer

In [ ]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np

# Load the BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Ensure the model is in evaluation mode
model.eval()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

#Step 2: Implement Functions to Get Word Embeddings and Perform Word Arithmetic

In [ ]:
# Function to get the embedding of a word
def get_word_embedding(word):
    # Tokenize the word and convert to tensor
    inputs = tokenizer(word, return_tensors='pt')

    # Get the hidden states from BERT
    with torch.no_grad():
        outputs = model(**inputs)

    # Use the embeddings from the last hidden layer, taking the average of the token embeddings
    word_embedding = outputs.last_hidden_state.mean(dim=1).squeeze()

    return word_embedding

# Function to perform word arithmetic
def word_arithmetic(words1):
    vec = get_word_embedding(words1[0])

    for i in range(1, len(words1), 2):
        if words1[i] == '-':
            vec -= get_word_embedding(words1[i + 1])
        elif words1[i] == '+':
            vec += get_word_embedding(words1[i + 1])

    return vec

# Function to find the most similar word
def find_most_similar(target_embedding, words_list):
    similarities = []

    for word in words_list:
        word_embedding = get_word_embedding(word)
        # Compute cosine similarity
        cos_sim = torch.nn.functional.cosine_similarity(target_embedding, word_embedding, dim=0)
        similarities.append((word, cos_sim.item()))

    # Sort based on similarity and return the most similar word
    similarities.sort(key=lambda x: x[1], reverse=True)

    return similarities[0][0]


#Step 3: Perform Word Arithmetic and Find Most Similar Word

In [ ]:
# List of examples
examples = [
    (["paris", "-", "france", "+", "italy"], ["rome", "romaine", "romania", "ronnie", "random"]),
    (["king", "-", "man", "+", "woman"], ["queen", "lady", "empress", "princess", "duchess"]),
    (["apple", "-", "fruit", "+", "vegetable"], ["carrot", "potato", "broccoli", "grape", "berry"]),
    (["japan", "-", "tokyo", "+", "seoul"], ["korea", "china", "singapore", "taiwan", "india"]),
    (["doctor", "-", "hospital", "+", "school"], ["teacher", "student", "principal", "nurse", "librarian"])
]

# Execute the word arithmetic for each example
for i, (arithmetic_words, candidates) in enumerate(examples):
    target_embedding = word_arithmetic(arithmetic_words)
    most_similar_word = find_most_similar(target_embedding, candidates)
    print(f"Example {i+1}: {arithmetic_words} -> Most similar word: {most_similar_word}")


Example 1: ['paris', '-', 'france', '+', 'italy'] -> Most similar word: rome
Example 2: ['king', '-', 'man', '+', 'woman'] -> Most similar word: queen
Example 3: ['apple', '-', 'fruit', '+', 'vegetable'] -> Most similar word: potato
Example 4: ['japan', '-', 'tokyo', '+', 'seoul'] -> Most similar word: korea
Example 5: ['doctor', '-', 'hospital', '+', 'school'] -> Most similar word: teacher


#Part 2: RAG System Implementation

In [1]:
!pip install langchain
!pip install langchain-community
!pip install langchain-groq
!pip install wikipedia
!pip install sentence-transformers
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 523.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.7/399.7 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.2/290.2 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/1

In [2]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI  # Assuming ChatGroq works similarly to OpenAI models
from langchain_groq import ChatGroq
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import WikipediaLoader
import os

# Set Groq API key
os.environ["GROQ_API_KEY"] = "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"

# Step 1: Choose 5 articles (article titles are just examples, adjust as needed)
articles = [
    "Artificial Intelligence",
    "Quantum Computing",
    "Climate Change",
    "Ancient Civilizations",
    "Space Exploration"
]

# Step 2: Load and process each article
documents = []
for article_title in articles:
    loader = WikipediaLoader(article_title)
    article_text = loader.load()
    documents.extend(article_text)

# Step 3: Split the documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs_chunks = text_splitter.split_documents(documents)

# Step 4: Create embeddings and store in a VectorDB
embedding_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
vector_db = FAISS.from_documents(docs_chunks, embedding_model)

# Step 5: Initialize the Groq LLM
llm = ChatGroq(model_name="mixtral-8x7b-32768")  # Ensure ChatGroq is compatible with langchain

# Step 6: Load the QA chain using the appropriate LLM
qa_chain = load_qa_chain(llm, chain_type="stuff")

# Step 7: Define the query function using the chain and retriever
def run_query(query):
    docs = vector_db.similarity_search(query)
    result = qa_chain.run(input_documents=docs, question=query)
    return result

# Step 8: Run 10 diverse queries on the RAG system
queries = [
    "What is the main goal of Artificial Intelligence?",
    "How does Quantum Computing differ from classical computing?",
    "What are the primary causes of Climate Change?",
    "Which ancient civilization was known for building pyramids?",
    "What is the significance of space exploration?",
    "How is AI impacting modern industries?",
    "What is the principle of superposition in Quantum Computing?",
    "What are the potential effects of climate change on the environment?",
    "Which ancient civilization is credited with the invention of the wheel?",
    "What are the future prospects of human settlement on Mars?"
]

# Step 9: Run each query and record results
for i, query in enumerate(queries, 1):
    response = run_query(query)
    print(f"Query {i}: {query}")
    print(f"Response: {response}\n")

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
<ipython-input-2-e6a492ba3f4d>:34: LangChainDeprec

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-2-e6a492ba3f4d>:41: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/v0.2/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/v0.2/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/v0.2/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/v0.2/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/v0.2/docs/how_to/#qa-with-rag
  qa_chain = load_qa_chain(llm, chain_type="stuff")
<ipython-input-2-e6a492ba3f4d>:46: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain.run(input_documents=docs, question=query)


Query 1: What is the main goal of Artificial Intelligence?
Response: The main goal of Artificial Intelligence is to create or simulate intelligence in machines, particularly computer systems, with the ability to perceive their environment, learn, and take actions that maximize their chances of achieving defined goals. This overarching goal is broken down into specific subproblems or traits that researchers aim to exhibit in intelligent systems, such as reasoning, knowledge representation, planning, learning, natural language processing, perception, and support for robotics. The ultimate long-term goal is to develop a system that possesses general intelligence, capable of completing any task performable by a human on at least an equal level.

Query 2: How does Quantum Computing differ from classical computing?
Response: Quantum computing differs from classical computing in several ways. First, a quantum algorithm, which runs on a quantum computer, can take advantage of quantum mechanica